In [1]:
import gsd.hoomd
import hoomd
import matplotlib
import numpy

%matplotlib inline
matplotlib.style.use('ggplot')
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
import os

fn = os.path.join(os.getcwd(), 'trajectory.gsd')
![ -e "$fn" ] && rm "$fn"

In [3]:
# This is not intended as a full tutorial on fresnel - see the fresnel user
# documentation (https://fresnel.readthedocs.io/) if you would like to learn more.

import math

import fresnel
import IPython

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=1000, h=1000)


def render(snapshot):
    L = snapshot.configuration.box[0]
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(
        scene, N=len(snapshot.particles.position), radius=99.5
    )
    geometry.material = fresnel.material.Material(
        color=fresnel.color.linear([2 / 255, 252 / 255, 1 / 255]),
        roughness=0.2,
        primitive_color_mix=1.0,
    )
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 10
    energies = snapshot.log['particles/md/pair/Mie/energies']
    mapper = matplotlib.cm.ScalarMappable(
        norm=matplotlib.colors.Normalize(
            vmin=-64, vmax=0, clip=True
        ),
        cmap=matplotlib.colormaps['viridis'],
    )
    geometry.color[:] = fresnel.color.linear(mapper.to_rgba(energies))

    fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0])

    scene.lights = [
        fresnel.light.Light(direction=(1,0,0), color=(0.8, 0.8, 0.8), theta=math.pi),
        fresnel.light.Light(
            direction=(1, 1, 0), color=(1.1, 1.1, 1.1), theta=math.pi / 3
        ),
    ]
    scene.camera = fresnel.camera.Orthographic(
        position=(L*2, L, L*2), look_at=(0, 0, 0), up=(0,1,0), height=L *1.2 + 1
    )
    scene.background_alpha = 0
    scene.background_color = (0, 0, 0)
    return IPython.display.Image(tracer.sample(scene, samples=1000)._repr_png_())

In [4]:
cpu = hoomd.device.CPU()
simulation = hoomd.Simulation(device=cpu, seed=1)
simulation.create_state_from_gsd(
    filename='/root/traj10_G4.gsd'
)

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer=100)
mie = hoomd.md.pair.Mie(nlist=cell)
mie.params[('A', 'A')] = dict(epsilon=9, sigma=209, n=45, m=18)
mie.r_cut[('A', 'A')] = 260

integrator.forces.append(mie)
nvt = hoomd.md.methods.ConstantVolume(
    filter=hoomd.filter.All(), thermostat=hoomd.md.methods.thermostats.Bussi(kT=2.5)
)
integrator.methods.append(nvt)
simulation.operations.integrator = integrator
simulation.run(1)

In [6]:
logger = hoomd.logging.Logger()
logger.add(mie, quantities=['energies', 'forces'])

In [7]:
gsd_writer = hoomd.write.GSD(
    filename='trajectory.gsd',
    trigger=hoomd.trigger.Periodic(10000),
    mode='xb',
    filter=hoomd.filter.All(),
)
simulation.operations.writers.append(gsd_writer)

In [8]:
gsd_writer.logger = logger

In [9]:
simulation.run(10000)

In [10]:
gsd_writer.flush()

In [11]:
traj = gsd.hoomd.open('trajectory.gsd', mode='r')

In [ ]:
traj[0].log.keys()

In [ ]:
traj[-1].log['particles/md/pair/Mie/energies'][1001:2000]

In [ ]:
fig = matplotlib.figure.Figure(figsize=(5, 3.09))
ax = fig.add_subplot()
ax.hist(traj[-1].log['particles/md/pair/Mie/energies'], 100)
ax.set_xlabel('potential energy')
ax.set_ylabel('count')
fig

In [ ]:
render(traj[-1])

[Previous section](01-Logging-to-a-file.ipynb) / [Next section](03-Storing-Particle-Shape.ipynb)